In [ ]:
#Install libiaries 
# !pip install hubspot-api-client
import webbrowser
from hubspot import HubSpot
import numpy as np
import pandas as pd
import pyodbc
import os
import json
from time import sleep
from datetime import datetime, timedelta
from hubspot.auth.oauth import ApiException
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
api_client = HubSpot()
api_client = HubSpot(api_key='**************************')
print("Finished Import")

In [ ]:
#Get Authorization Code
webbrowser.open('VIVBI.com/hubspot')  # Goes to the URL for a user to authenticate 
#After the User authenticates they will be redirected to VIVBI.com/?code=XXXXXXXX-XXXX-XXXX-XXXXXXXXXXXX
##capture that authorization code. 
print("Get key from VIVBI.com")

fAuthCode = input('Copy key in from website after the "code="',)

try:
    tokens = api_client.auth.oauth.default_api.create_token(
        grant_type="authorization_code",
        redirect_uri='https://vivbi.com',
        client_id='************************',
        client_secret='*****************************',
        code=fAuthCode
    )
except ApiException as e:
    print("Exception when calling create_token method: %s\n" % e)

print("Token and refresh token recieved ",datetime.today())

api_client = HubSpot()
api_client.access_token = tokens.access_token
print("Client Configured, access token expires in ", datetime.now() + timedelta(seconds=tokens.expires_in))
with open('refresh_token.txt', 'w') as f:
    f.write(tokens.refresh_token)
    print("refresh_token.txt created")

In [ ]:
#Gets new access token with refresh token from file.
with open('refresh_token.txt','r') as f:
    contents = f.read()

try:
    tokens = api_client.auth.oauth.default_api.create_token(
        grant_type="refresh_token",
        redirect_uri='https://vivbi.com',
        client_id='**************',
        client_secret='***************',
        refresh_token=contents
    )
except ApiException as e:
    print("Exception when calling create_token method: %s\n" % e)

api_client = HubSpot()
api_client.access_token = tokens.access_token

print("Client Configured, access token expires at ", datetime.now() + timedelta(seconds=tokens.expires_in))
with open('refresh_token.txt', 'w') as f:
    f.write(tokens.refresh_token)
    print("refresh_token.txt updated")
print(tokens.access_token)

In [ ]:
#Creates Contact in CRM for testing
from hubspot.crm.contacts import SimplePublicObjectInput
from hubspot.crm.contacts.exceptions import ApiException
for number in range(4000):
    #print(number)
    try:
        simple_public_object_input = SimplePublicObjectInput(
            properties={"email": "test__"+str(number+1500)+"@vivbi.com"}
        )
        api_response = api_client.crm.contacts.basic_api.create(
            simple_public_object_input=simple_public_object_input
        )
    except ApiException as e:
        print("Exception when creating contact: %s\n" % e)
    sleep(1)


In [ ]:
#Gets all contacts from hubspot and creates a dataframe called contacts_df
all_contacts = api_client.crm.contacts.get_all()
id_values = []
email = []
firstname = []
lastname = []
lastmodifieddate = []
created_at = []
archived=[]
contact_assocations=[]
for contact in all_contacts:
    id_values.append(contact.id) #0
    created_at.append(contact.properties['createdate']) #1
    firstname.append(contact.properties['firstname']) #2
    lastname.append(contact.properties['lastname'])#3
    lastmodifieddate.append(contact.properties['lastmodifieddate'])#4
    archived.append(contact.archived) #5
    email.append(contact.properties['email'])
    contact_assocations.append(contact.associations)


contacts_df = pd.DataFrame(id_values,columns=['id'])
contacts_df.insert(1, "created_at", created_at, True)
contacts_df.insert(2,"firstname",firstname,True)
contacts_df.insert(3,"lastname",lastname,True)
contacts_df.insert(4,"lastmodifieddate",lastmodifieddate,True)
contacts_df.insert(5,"archived",archived,True)
contacts_df.insert(6,"email",email,True)
contacts_df.insert(7,"associations",contact_assocations,True)
print("Hubspot contact import complete",datetime.now())
contacts_df

In [ ]:
#Gets all deals from hubspot and creates a dataframe called deals_df
all_deals = api_client.crm.deals.get_all()
deal_id=[]
deal_archived=[]
deal_created_at=[]
deal_closeddate=[]
deal_name=[]
deal_dealstage=[]
deal_lastmodifieddate=[]
deal_pipeline=[]
deal_associations=[]
for deal in all_deals:
    deal_id.append(deal.id) #0
    deal_archived.append(deal.archived)#
    deal_created_at.append(deal.created_at)#1
    deal_closeddate.append(deal.properties['closedate'])#2
    deal_name.append(deal.properties['dealname'])#3
    deal_dealstage.append(deal.properties['dealstage'])#5
    deal_lastmodifieddate.append(deal.properties['hs_lastmodifieddate'])#6
    deal_pipeline.append(deal.properties['pipeline'])#7
    deal_associations.append(deal.associations)

deals_df = pd.DataFrame(deal_id,columns=['id']) #0
deals_df.insert(1, "deal_created_at", deal_created_at, True)
deals_df.insert(2,'deal_closeddate',deal_closeddate,True)
deals_df.insert(3,'deal_name',deal_name,True)
deals_df.insert(4,'deal_dealstage',deal_dealstage,True)
deals_df.insert(5,'archived',deal_archived,True)
deals_df.insert(6,'deal_lastmodifieddate',deal_lastmodifieddate,True)
deals_df.insert(7,'deal_pipeline',deal_pipeline,True)
deals_df.insert(8,'associations',deal_associations,True)

print("Hubspot deal import complete",datetime.now())
deals_df

In [ ]:
#Gets all companies from hubspot and creates a dataframe called deals_df
all_companies = api_client.crm.companies.get_all()
companies_id=[]
companies_created_at=[]
companies_archived=[]
companies_lastmodifieddate=[]
companies_domain=[]
companies_name=[]
companies_associations=[]
for companies in all_companies:
    companies_id.append(companies.id) #0
    companies_created_at.append(companies.created_at)#1
    companies_archived.append(companies.archived) #2
    companies_lastmodifieddate.append(companies.properties['hs_lastmodifieddate'])#3
    companies_domain.append(companies.properties['domain'])#4
    companies_name.append(companies.properties['name'])#5
    companies_associations.append(companies.associations)#6

companies_df = pd.DataFrame(companies_id,columns=['id']) #0
companies_df.insert(1, "companies_created_at", companies_created_at, True) #1
companies_df.insert(2,"companies_archived",companies_archived,True)
companies_df.insert(3,"companies_lastmodifieddate",companies_lastmodifieddate,True)
companies_df.insert(4,"companies_domain",companies_domain,True)
companies_df.insert(5,"companies_name",companies_name,True)
companies_df.insert(6,"companies_associations",companies_associations,True)

print("Hubspot companies import complete",datetime.now())
companies_df